In [1]:
import torch 
import torch.nn as nn 
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader, Dataset 
from torchvision.datasets import ImageFolder 
import torchvision.models as models 
import torch.optim as optim

In [2]:
transform = transforms.Compose([ 
transforms.Resize((224, 224)), 
transforms.ToTensor(), 
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
]) 

In [12]:
class SkinDefectsDataset(Dataset): 
    def __init__(self, data_dir, transform=None): 
        self.data = ImageFolder(data_dir, transform=transform) 
    def __len__(self): 
        return len(self.data)   
    def __getitem__(self, idx):
        img, label = self.data[idx]
        # Convert label to one-hot encoding
        label = torch.zeros(5).scatter_(0, torch.tensor(label), 1)
        return img, label 

In [13]:
train_dataset = SkinDefectsDataset("dataset/train", transform=transform) 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) 

In [14]:
model = models.resnet50(pretrained=True) 
num_features = model.fc.in_features 

c:\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
model.fc = nn.Sequential( 
nn.Linear(num_features, 5),  # 5 output nodes for 5 defect types 
nn.Sigmoid()  # Sigmoid activation for multi-label classification 
)

In [16]:
criterion = nn.BCELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001) 
num_epochs = 10

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [18]:
device

device(type='cpu')

In [19]:
model.to(device) 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
for epoch in range(num_epochs): 
    model.train() 
    running_loss = 0.0 
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().to(device) 
        optimizer.zero_grad() 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_dataset) 
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}') 

Epoch [1/10], Loss: 0.4057
Epoch [2/10], Loss: 0.3437
Epoch [3/10], Loss: 0.3193
Epoch [4/10], Loss: 0.2802
Epoch [5/10], Loss: 0.2558
Epoch [6/10], Loss: 0.2173
Epoch [7/10], Loss: 0.2048
Epoch [8/10], Loss: 0.1826
Epoch [9/10], Loss: 0.1702
Epoch [10/10], Loss: 0.1214


In [21]:
torch.save(model.state_dict(), 'skin_defects_model.pth')